In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
from micrograd.engine import Value
from micrograd.nn import MLP

In [3]:
df = pd.read_csv("./data/stock.csv")
df["target"] = [1 if df["close"][i + 1] > df["close"][i] else 0 for i in range(len(df) - 1)] + [0]
predictors = ["Trading_Volume", "Trading_money", "open", "max", "min", "close", "spread", "Trading_turnover"]
x_train = df[predictors].iloc[:-100].to_numpy()
x_test = df[predictors].iloc[-100:].to_numpy()
y_train = df["target"].iloc[:-100].to_numpy()
y_test = df["target"].iloc[-100:].to_numpy()
y_train = keras.utils.to_categorical(y_train)

In [4]:
epochs = 6
batch_size = 32
learning_rate = 0.3
model = MLP(8, [64, 64, 2])

print(model)
print("number of parameters", sum(p.data.size for p in model.parameters()))

MLP of [ReLULayer(8, 64), ReLULayer(64, 64), LinearLayer(64, 2)]
number of parameters 4866


In [5]:
for epoch in range(1, epochs + 1):
    pre = 0
    cur = batch_size
    index = np.arange(x_train.shape[0])

    while pre < cur:
        ri = index[pre:cur]
        Xb, yb = Value(x_train[ri]), Value(y_train[ri])

        y_predW = model(Xb)
        probs = y_predW.softmax()
        log_probs = probs.log()
        zb = yb * log_probs
        outb = zb.reduce_sum(axis = 1)
        finb = -outb.reduce_sum()
        loss = finb / (cur - pre)

        model.zero_grad()
        loss.backward()

        for p in model.parameters():
            if p.data.ndim > 1:
                p.data -= learning_rate * p.grad
            else:
                p.data -= learning_rate * p.grad.sum(axis = 0)

        pre = cur

        if cur <= x_train.shape[0] - batch_size:
            cur += batch_size
        else:
            cur = x_train.shape[0]

    print(f'epoch {epoch}, accuracy: {(accuracy_score(np.argmax(model(x_test).data, axis = 1), y_test) * 100):.2f}%')

epoch 1, accuracy: 54.00%
epoch 2, accuracy: 54.00%
epoch 3, accuracy: 54.00%
epoch 4, accuracy: 54.00%
epoch 5, accuracy: 54.00%
epoch 6, accuracy: 54.00%
